In [1]:
# main URL area, contain for each novel 
url = "https://www.xbiquge.so/book/49549/"

In [19]:
# import area
import requests
from bs4 import BeautifulSoup

In [80]:
# Parse the html text and retreive all the chapters 
r = requests.get(url)
soup = BeautifulSoup(r.text)

# Since BiQuGe use cache in their website, we need to drop first 12 website
# which are the newest 12 chapters and have been included later
sub_url_list = soup.find(id='list').find_all('a')[12:]
sub_url_list = [sub_url.get("href") for sub_url in sub_url_list]
title = str(soup.find(id='list').find("dt")).split("》")[0].split("《")[1]

In [107]:
# By checking the corresponding website to retrieve content
def get_text_and_title(url, sub_url):
#     sub_url = sub_url_list[0]
    chapter_url = url + sub_url
    r = requests.get(chapter_url)
    soup = BeautifulSoup(r.text)
    sub_text = soup.find(id='content').text.split("\xa0\xa0\xa0\xa0")[1:]
    sub_title = soup.find("title").text.split("_")[0]
    return sub_text, sub_title

In [87]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="webkit" name="renderer"/>
  <title>
   第一章 我只想当个窝囊废_豪婿-笔趣阁
  </title>
  <link href="https://www.xbiquge.so/book/49549/31073090.html" rel="canonical"/>
  <meta content="第一章 我只想当个窝囊废,豪婿" name="keywords"/>
  <meta content="豪婿最新章节第一章 我只想当个窝囊废全文免费阅读，笔趣阁精选豪婿无错最新章节。" name="description"/>
  <meta content="format=html5;url=https://m.xbiquge.so/book/49549/31073090.html" name="mobile-agent"/>
  <meta content="format=xhtml;url=https://m.xbiquge.so/book/49549/31073090.html" name="mobile-agent"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
  <meta content="pc" name="applicable-device"/>
  <meta content="no-transform" http-equiv="Cache-Control"/>
  <meta content="no-siteapp" http-equiv="Cache-Control"/>
  <link href="/biquge/css/common.css" rel="stylesheet" type="text/css"/>
  <link href="/biquge